# Overdue HW 4 (oops :[ )


## Exercises

You may work in groups for these exercises. They are due at the beginning of next class. You can submit them as a link to a Colab notebook or GitHub CodeSpace.

0. Use the corpora that you assembled last week (Pausanias++):
1. Using programming techniques from the course so far, find other potential collocates for a word of your choice.
2. Calculate the μ and Mutual Information scores for at least 5 of these collocate pairs. How do your results change depending on your definition of a collocation? What might these changes mean? (Write your answers to these questions down.)
3. Calculate the Delta P for these same five pairs. Do any results stand out? Why? What might they tell us about your corpus.


## 0

In [1]:
%pip install pip==24.0
%pip install -r requirements.txt
%run -m grecy install grc_proiel_sm


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached spacy-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
Reason for being yanked: model compatibility problem
Using cached spacy-3.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.4 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grc-proiel-sm 3.7.5 requires spacy<3.8.0,>=3.7.5, but you have spacy 3.8.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel 

In [2]:
from lxml import etree
from MyCapytain.common.constants import Mimetypes
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
import pandas as pd

with open("tlg0525.tlg001.perseus-grc2.xml") as f:
    text = CapitainsCtsText(urn="urn:cts:greekLit:tlg0525.tlg001.perseus-grc2", resource=f)

urns = []
raw_xmls = []
unannotated_strings = []

for ref in text.getReffs(level=len(text.citation)):
    urn = f"{text.urn}:{ref}"
    node = text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
pausanias_df = pd.DataFrame(d)
pausanias_df['whitespaced_tokens'] = pausanias_df['unannotated_strings'].str.split()
#pausanias_df

In [3]:
# this will take a while

import spacy

nlp = spacy.load("grc_proiel_sm", disable=["ner"])

raw_texts = [t for t in pausanias_df['unannotated_strings']]
annotated_texts = nlp.pipe(raw_texts, batch_size=100)

pausanias_df['nlp_docs'] = list(annotated_texts)

## 1
Using programming techniques from the course so far, find other potential collocates for a word of your choice. 

In [ ]:
from collections import Counter

def lemma_tize(c):
    return [b.lemma_ for b in c]

test_token = pausanias_df['nlp_docs'][0][1]
pausanias_df['my_lemmata'] = [lemma_tize(c) for c in pausanias_df['nlp_docs']]

# we use a list comprehension to evaluate the generator at `test_token.children`
f"token: '{test_token}, {test_token.lemma_}', dependencies: {[(c, c.lemma_) for c in test_token.children]}"

# λόγος time
#homer_son_lines = homer_df.loc[homer_df["unannotated_strings"].str.contains('son', regex=False)]
paus_son_lines = pausanias_df.loc[pausanias_df["my_lemmata"].str.contains('λόγος', regex=False)]

collocates = []
for x in paus_son_lines['nlp_docs']:
    for y in x:
        if y.lemma_ == 'λόγος':
            collocates.extend([c.lemma_ for c in y.children])
        else:
            if 'λόγος' in [d.lemma_ for d in y.children]:
                collocates.append((y.lemma_))

cnt_locates = Counter(collocates)
cnt_locates.most_common(20)
# sarah's five collocates: ἔχω, δηλόω, λέγω, ποιέω, εἰμί

[('ὁ', 245),
 (',', 49),
 ('ἔχω', 34),
 ('ἐν', 31),
 ('δηλόω', 22),
 ('λέγω', 20),
 ('ποιέω', 20),
 ('εἰς', 19),
 ('ἐπί', 17),
 ('εἰμί', 16),
 ('οὗτος', 15),
 ('αὐτός', 14),
 ('οὐδείς', 13),
 ('καί', 12),
 ('δέ', 12),
 ('κατά', 10),
 ('ἐγώ', 9),
 ('ἀληθής', 8),
 ('·', 8),
 ('ἕτερος', 8)]

## 2
Calculate the μ and Mutual Information scores for at least 5 of these collocate pairs. How do your results change depending on your definition of a collocation? What might these changes mean? (Write your answers to these questions down.)

In [ ]:
# sarah's five collocates: ἔχω, δηλόω, λέγω, ποιέω, εἰμί



## 3
Calculate the Delta P for these same five pairs. Do any results stand out? Why? What might they tell us about your corpus.